In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
from republic.elastic.republic_elasticsearch import initialize_es

rep_es = initialize_es(host_type='external', timeout=60)


In [3]:
from collections import Counter
import re

query = {
    "bool": {
        "must": [
            {"match": {"type": "resolution"}},
            {"match": {"metadata.session_year": 1672}}
        ]
    }
}

resolutions_1672 = rep_es.retrieve_resolutions_by_query(query, size=10000)



In [31]:
from collections import defaultdict

# Create a counting object
unigram_freq = Counter()
bigram_freq = Counter()
prefix_freq = defaultdict(Counter)
postfix_freq = defaultdict(Counter)

# iterate over all resolutions and ...
for res in resolutions_1672:
    # ... over all paragraphs in a resolution
    for para in res.paragraphs:
        # and count the occurrence of each word
        unigrams = [word for word in re.split(r"\W+", para.text) if word != '']
        bigrams = [f"{unigrams[i]} {unigrams[i+1]}" for i, w in enumerate(unigrams[:-1])]
        unigram_freq.update(unigrams)
        bigram_freq.update(bigrams)
        for i, w1 in enumerate(unigrams[:-1]):
            w2 = unigrams[i+1]
            prefix_freq[w2].update([w1])
            postfix_freq[w1].update([w2])

# show the frequency of the 10 most frequent words
for unigram, freq in unigram_freq.most_common(10):
    print(f"{unigram: <20}{freq: >6}")

ende                 26333
van                  23211
de                   20743
te                   14955
dat                  11138
den                   9356
haer                  8467
vande                 8418
tot                   8380
in                    8012


In [19]:
# show the frequency of the 10 most frequent words
for bigram, freq in bigram_freq.most_common(10):
    print(f"{bigram: <20}{freq: >6}")

Ho Mo                 7155
haer Ho               5225
ende verstaen         2851
dat de                2665
de voorschreve        2587
sal werden            2346
verstaen dat          2262
goetgevonden ende     2182
Waerop gedelibereert  2167
gedelibereert sijnde  2028


In [21]:
from republic.helper.text_helper import SkipgramSimilarity

unigram_sim = SkipgramSimilarity(ngram_length=2, skip_length=2, terms=unigram_freq.keys())


In [34]:
for t, s in unigram_sim.rank_similar('becommerlijcke', top_n=20):
    print(f"{t: <30}{unigram_freq[t]: >5}{s: 6.2f}")

becommerlijcke                    5  1.00
becommerlijcken                   1  0.97
commerlijcke                      1  0.93
becommerlicke                     7  0.89
becommerlicken                    3  0.86
beecomerlijcken                   1  0.84
becommeringe                      3  0.68
onbecommerden                     2  0.62
becommernisse                     1  0.62
behoerlijcke                      1  0.61
merckelijcke                     11  0.58
commercieren                      1  0.56
merckelijcken                     3  0.56
baerblijckelijck                  1  0.54
baerblijckelicke                  1  0.53
keijserlijcke                     1  0.52
Keijserlijcke                     3  0.51
gebruijckelijcke                  1  0.51
gelijckelick                      1  0.50
beswaerlijcke                     2  0.50


In [32]:
prefix_freq['becommerlijcke'].most_common(10)

[('dese', 4), ('tegenwoordige', 1)]

In [33]:
postfix_freq['becommerlijcke'].most_common(10)

[('tijden', 2), ('toestant', 1), ('constitutie', 1), ('conuuoture', 1)]

In [24]:
bigram_sim = SkipgramSimilarity(ngram_length=2, skip_length=2, terms=bigram_freq.keys())


In [28]:
bigram_sim.rank_similar('sware tijden', top_n=20)

[('ter tijden', 0.7080881928749334),
 ('dese tijden', 0.648074069840786),
 ('vande tijden', 0.6454972243679028),
 ('onder tijden', 0.6454972243679028),
 ('anderen tijden', 0.6324555320336759),
 ('dien tijde', 0.6210590034081187),
 ('ouden tijden', 0.6085806194501846),
 ('tijden wert', 0.5973191135873606),
 ('ware sijne', 0.5962847939999439),
 ('van tijden', 0.5962847939999439),
 ('sware diensten', 0.5923488777590924),
 ('Vredens tijden', 0.5797509043642028),
 ('tijden ten', 0.5728918992315464),
 ('tijden tot', 0.5728918992315464),
 ('tijden sij', 0.5728918992315464),
 ('bij tijden', 0.5728918992315464),
 ('daertoe bijden', 0.5627314338711378),
 ('allen tijden', 0.5627314338711378),
 ('slechte tijden', 0.5627314338711378),
 ('hare zijde', 0.5590169943749475)]

In [26]:
for hit in rep_es.keyword_in_context("tijden", num_hits=20, context_size=1):
    print(hit["context"])


            in tijden van
       gesette tijden, van
       sware - tijden van
          dese tijden van
becommerlijcke tijden te
          dese tijden niet
         allen tijden van
            in tijden van
           van tijden ende
           van tijden en
           van tijden ende
         allen tijden, als
           van tijden sigh
         allen tijden, soo
overdenckelijcke tijden, de
           van tijden Præside
    voorgaende tijden van
           van tijden ende
           van tijden, ten
       trouble tijden met
      perplexe tijden 't
       voorige tijden tot
         allen tijden, de
          dese tijden te


In [4]:
def json_to_csv(data):
    return [data[field] for field in data]


def get_resolution_text(resolution, join_char: str = ' '):
    return join_char.join([para.text for para in resolution.paragraphs])


def resolution_to_csv(resolution, headers):
    row = []
    for header in headers:
        if header == "text":
            text = get_resolution_text(resolution)
            row.append(text)
        elif hasattr(resolution, header):
            row.append(getattr(resolution, header))
        elif header in resolution.metadata:
            row.append(resolution.metadata[header])
        else:
            row.append(None)
    return row
    

for resolution in resolutions_1672:
    res_row = resolution_to_csv(resolution, ['id', 'session_date', 'proposition_type', 'text'])
    print(res_row)
    break

['session-1672-01-07-num-1-resolution-1', RepublicDate(1672-01-07), None, "Is gehoort het rapport vande Heeren Schimmelpenningh, ende andere hare Ho:Mo: Gedeputeerden tot de saken vande Zee, hebbende ingevolge ende tot voldoeninge van derselver resolutie Commissoriael vanden negenentwin„ tichsten December laestleden, gevi, siteert ende geexamineert de Requeste van David Centsen, Consul vande Nederlantsche natie tot Rochelle, versoeckende door hare Ho:Mo: met eene somme van penningen te mogen werden gesubvenieert, ten aensien vande oncosten bij hem gesupporteert in een continueel vervolgh van ontrent acht maenden, om expeditie, en het obtineren van eene resolutie op de Consulaetrechten aldaer, ende een daghgelt aen hem Suppliant als Con„ sul toe te leggen: Waerop gedelibereert sijnde, Is goetgevonden ende verstaen, mits desen te versoec„ ken de Heeren Gedeputeerden vande Provincie van Hollandt ende West,, vrieslandt, dat haer E. haer hoe eerder soo liever willen verclaren op het rapport

In [4]:
from fuzzy_search.fuzzy_context_searcher import FuzzyContextSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel


def get_resolution_text(resolution, join_char: str = ' '):
    """Return all paragraphs of a resolution joined into a single text string."""
    return join_char.join([para.text for para in resolution.paragraphs])


def resolution_to_csv(resolution, headers):
    """Extra fields from a resolution given a list of headers."""
    row = []
    for header in headers:
        if header == "text":
            text = get_resolution_text(resolution)
            row.append(text)
        elif header == "session_date":
            row.append(resolution.session_date.isoformat())
        elif hasattr(resolution, header):
            value = getattr(resolution, header)
            row.append(value if value is not None else '')
        elif header in resolution.metadata:
            row.append(resolution.metadata[header])
        else:
            row.append('')
    return row
    


phrases = [
    {
        "phrase": "tijden",
        "distractors": [
            "lijden", 'Leijden', 'huijden'
        ]
    }
]

config = {
    'levenshtein_threshold': 0.7,
    'ngram_size': 3,
    'skip_size': 1,
    'include_variants': True,
    'filter_distractors': True
}

phrase_model = PhraseModel(phrases, config=config)
searcher = FuzzyContextSearcher(config)
searcher.index_phrase_model(phrase_model)

# fields to extract from the matches
match_headers = [
    'phrase', 'match_string', 'match_context'
]
# fields to extract from the resolutions
res_headers = [
    'id', 'session_date', 'proposition_type', 'text'    
]

# the set of all column headers for the output file
all_headers = res_headers + match_headers

# start with an empty list for collecting rows of matches
rows = []

output_file = "tijden_matches.csv"

with open(output_file, 'wt') as fh:
    print(all_headers)
    print()
    row = '\t'.join(all_headers)
    fh.write(f"{row}\n")
    for res in resolutions_1672[:200]:
        print(res.id, res.proposition_type)
        for para in res.paragraphs:
            matches = searcher.find_matches(para.text)
            res_row = resolution_to_csv(res, res_headers)
            for match in matches:
                match_row = res_row + [match.phrase.phrase_string, match.string, match.context]
                print(res_row)
                print(match_row)
                row = '\t'.join(match_row)
                fh.write(f"{row}\n")

['id', 'session_date', 'proposition_type', 'text', 'phrase', 'match_string', 'match_context']

session-1672-01-07-num-1-resolution-1 None
session-1672-01-07-num-1-resolution-2 missive
session-1672-01-07-num-1-resolution-3 requeste
session-1672-01-07-num-1-resolution-4 missive
session-1672-01-07-num-1-resolution-5 missive
session-1672-01-07-num-1-resolution-6 missive
session-1672-01-07-num-1-resolution-7 missive
session-1672-01-07-num-1-resolution-8 missive
session-1672-01-07-num-1-resolution-9 missive
session-1672-01-07-num-1-resolution-10 None
session-1672-01-07-num-1-resolution-11 None
session-1672-01-12-num-1-resolution-1 missive
session-1672-01-12-num-1-resolution-2 missive
session-1672-01-12-num-1-resolution-3 missive
session-1672-01-12-num-1-resolution-4 missive
session-1672-01-12-num-1-resolution-5 None
session-1672-01-12-num-1-resolution-6 None
session-1672-01-12-num-1-resolution-7 missive
session-1672-01-12-num-1-resolution-8 missive
session-1672-01-12-num-1-resolution-9 missi

session-1672-05-13-num-1-resolution-3 missive
session-1672-05-13-num-1-resolution-4 missive
session-1672-05-13-num-1-resolution-5 missive
session-1672-05-13-num-1-resolution-6 missive
['session-1672-05-13-num-1-resolution-6', '1672-05-13', 'missive', "Ontfangen een Missive vanden Heere Grave van Dhona, geschreven tot Lurich den derden deses, houdende advertentie: Waerop geen resolutie en is gevallen. De Heeren Gedeputeerden vande Pro„ vincie van Vrieslandt hebben ten Vergaderinge geopent de provinciale resolutien vande Heeren Staten vande hoochgemelte Provincie, tot bevorde, ringe vande gemeijne saecke ge,, nomen, hier naer van woorde te woorde geinsereert: Waerop geen resolutie en is gevallen. Oopia Gehoort de propositie vande heeren haer Ho:Mo: Gedeputeerden den drijentwintichsten deses in volle Vergaderinge gedaen, hebben haer Ed:Mo: opt eerste poinct, raec„ kende het werven ende aenstellen van derselver quote in de thien duijsent Waertgelders, goetgevonden te inhereren haer voorigh

session-1672-05-16-num-1-resolution-9 None
session-1672-05-17-num-1-resolution-1 missive
session-1672-05-17-num-1-resolution-2 missive
session-1672-05-17-num-1-resolution-3 missive
session-1672-05-17-num-1-resolution-4 missive
session-1672-05-17-num-1-resolution-5 missive
session-1672-05-17-num-1-resolution-6 requeste
session-1672-05-17-num-1-resolution-7 missive
session-1672-05-17-num-1-resolution-8 missive
session-1672-05-17-num-1-resolution-9 missive
session-1672-05-17-num-1-resolution-10 missive
session-1672-05-17-num-1-resolution-11 missive
session-1672-05-17-num-1-resolution-12 missive
session-1672-05-17-num-1-resolution-13 missive
session-1672-05-18-num-1-resolution-1 missive
session-1672-05-18-num-1-resolution-2 missive
session-1672-05-18-num-1-resolution-3 missive
session-1672-05-18-num-1-resolution-4 missive
session-1672-05-18-num-1-resolution-5 missive
session-1672-05-18-num-1-resolution-6 missive
['session-1672-05-18-num-1-resolution-6', '1672-05-18', 'missive', "Ontfangen 

session-1672-05-20-num-1-resolution-7 missive
session-1672-05-20-num-1-resolution-8 requeste
session-1672-05-20-num-1-resolution-9 missive
session-1672-05-20-num-1-resolution-10 missive
session-1672-05-20-num-1-resolution-11 missive
session-1672-05-20-num-1-resolution-12 None
session-1672-05-20-num-1-resolution-13 None
session-1672-05-20-num-1-resolution-14 None
['session-1672-05-20-num-1-resolution-14', '1672-05-20', '', "Is gehoort het rapport vande heeren van Brakell, ende andere hare Ho:Mo: Gedeputeerden tot de militaire saecken, hebbende in„ gevolge ende tot voldoeninge van derselver resolutie Commissoriael vanden negenthienden deses, gevi„ siteert ende geexamineert de requeste vande Ingesetenen vande vier Quartieren vande Meijerije van shertogenbosch, houdende dat tot haer kennisse gecomen sijnde, seeckere gedruckte Waerschouwinge, waerbij uijt den naem ende van wegen haer Ho:Mo: door Ge,, committeerden uijt den Raet van State in de geheele Meijerije van ‛s Hertogen„ bosch voorno

session-1672-05-24-num-1-resolution-2 missive
session-1672-05-24-num-1-resolution-3 missive
session-1672-05-24-num-1-resolution-4 missive
session-1672-05-24-num-1-resolution-5 missive
session-1672-05-24-num-1-resolution-6 missive
session-1672-05-24-num-1-resolution-7 missive
session-1672-05-24-num-1-resolution-8 None
session-1672-05-24-num-1-resolution-9 None
session-1672-05-24-num-1-resolution-10 missive
session-1672-05-24-num-1-resolution-11 missive
session-1672-05-24-num-1-resolution-12 requeste
session-1672-05-24-num-1-resolution-13 requeste
session-1672-05-24-num-1-resolution-14 None
session-1672-05-24-num-1-resolution-15 missive
session-1672-05-24-num-1-resolution-16 missive
session-1672-05-24-num-1-resolution-17 missive
session-1672-05-24-num-1-resolution-18 missive
session-1672-05-27-num-1-resolution-1 missive
session-1672-05-27-num-1-resolution-2 missive
session-1672-05-27-num-1-resolution-3 missive
session-1672-05-27-num-1-resolution-4 missive


In [4]:
import republic.model.resolution_phrase_model as rpm

from republic.parser.logical.pagexml_resolution_parser import make_resolution_phrase_model_searcher

res_searcher = make_resolution_phrase_model_searcher()
res_searcher.variants

adding phrases from set proposition_opening_phrases
adding phrases from set proposition_reason_phrases
adding phrases from set proposition_closing_phrases
adding phrases from set proposition_from_phrases
adding phrases from set proposition_verbs
adding phrases from set decision_phrases
adding phrases from set resolution_link_phrases
adding phrases from set prefix_phrases
adding phrases from set organisation_phrases
adding phrases from set location_phrases
adding phrases from set esteem_titles
adding phrases from set person_role_phrases
adding phrases from set military_phrases
adding phrases from set misc
adding phrases from set provinces
building phrase model for 250 resolution phrases


{Phrase(Advocaat voor, None),
 Phrase(DE Conclusie van eisen, None),
 Phrase(DE Resolutien eergisteren genomen, None),
 Phrase(DE Resolutien gisteren ge-, None),
 Phrase(DE Resolutien voorleede , None),
 Phrase(De resolutien eergisteren genomen, None),
 Phrase(De resolutien gisteren ge-, None),
 Phrase(De resolutien voorleden , None),
 Phrase(IS naar voorgaande deliberatie goedgevonden ende verstaan, None),
 Phrase(IS ter Vergaderinge geëxhibeert een Resolutie van, None),
 Phrase(Is gehoort het rapport vande , None),
 Phrase(Koopluyden, None),
 Phrase(Koopvrouw, None),
 Phrase(ONtfangen twee Missiven van , None),
 Phrase(Ontfangen een missive vanden , None),
 Phrase(Op de requeste van , None),
 Phrase(Regenten, None),
 Phrase(WAAR op geen resolutie voor alsnoch is gevallen, None),
 Phrase(Waer op geen resolutie is gevallen, None),
 Phrase(Waer op geen resolutie voor alsnoch is gevallen, None),
 Phrase(de Heer, None),
 Phrase(de Heeren, None),
 Phrase(de gewoonlijke Nieuwejaars-Gifte va

In [6]:
for res in resolutions_1672[:40]:
    if res.proposition_type is not None:
        continue
    #for match in res.evidence:
    #    print(match)
    for para in res.paragraphs[:1]:
        print(para.text)
        matches = res_searcher.find_matches(para.text)
        for match in matches:
            if 'proposition_opening' in match.label:
                print('\t', match.phrase.phrase_string, match.string, match.label)
        print()

Is gehoort het rapport vande Heeren Schimmelpenningh, ende andere hare Ho:Mo: Gedeputeerden tot de saken vande Zee, hebbende ingevolge ende tot voldoeninge van derselver resolutie Commissoriael vanden negenentwin„ tichsten December laestleden, gevi, siteert ende geexamineert de Requeste van David Centsen, Consul vande Nederlantsche natie tot Rochelle, versoeckende door hare Ho:Mo: met eene somme van penningen te mogen werden gesubvenieert, ten aensien vande oncosten bij hem gesupporteert in een continueel vervolgh van ontrent acht maenden, om expeditie, en het obtineren van eene resolutie op de Consulaetrechten aldaer, ende een daghgelt aen hem Suppliant als Con„ sul toe te leggen: Waerop gedelibereert sijnde, Is goetgevonden ende verstaen, mits desen te versoec„ ken de Heeren Gedeputeerden vande Provincie van Hollandt ende West,, vrieslandt, dat haer E. haer hoe eerder soo liever willen verclaren op het rapport vande gemelte Heeren Schimmel„ penningh ende andere hare Ho:Mo: Gedeputeer

	 OP de Requeste van  Op de requeste van  ['proposition_type:request', 'proposition_opening']

De requeste van Buijrschap Bokeles gelegen in het Stift van Munster in Westphalen, op de grensen van Oostvrieslant, ende der selver Ingesetenen, mitsgaders die van het huijs Esterwegen mede in het voor ss. Stift Munster aen Hummelingh gelegen, versoeckende Sauveguarde van haer Ho:Mo: Is naer voorgaende deliberatie goetgevonden ende verstaen, dat gestelt sal werden in handen vandeheeren van Brakel ende andere haer Ho:Mo: Gedeputeerden tot de militaire saecken, met ende nevens eenige heeren Gecommitteerden uijt den den Raet van State bij haer E. selfs te nomineren, om te visiteren, examineren, ende daer van rapport te doen.
	 De requeste van  De requeste van  ['proposition_type:requeste', 'proposition_opening']



In [8]:
print(res_searcher.phrase_model.custom["IS gehoort het rapport van "])

{'phrase': 'IS gehoort het rapport van ', 'variants': ['Is gehoort het rapport vande '], 'label': ['proposition_opening', 'proposition_type:rapport'], 'proposition_type': 'rapport', 'max_offset': 10}
